In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/classification_dataset/Train'
valid_path = '/content/drive/MyDrive/classification_dataset/Test'


In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/classification_dataset/Train/*')
len(folders)

In [ ]:
folders

In [ ]:
# our layers 
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
steps_per_epoch=len(training_set),    
  validation_steps=len(test_set) 
  
)

In [ ]:
test_loss,test_acc=model.evaluate(test_set)

print('{:#.3g}'.format(test_acc),' : Is the accuracy \n')
print(np.round(test_loss,2),':  Is the Loss')

In [ ]:
r.history.keys()

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
##  saving and Loading the model
from keras.models import load_model

model.save('Coins_Classification_model1_vgg16.h5')

# New Section

# New Section

In [ ]:
from tensorflow.keras.models import load_model
Vgg_16_model=load_model('Coins_Classification_model1_vgg16.h5')

In [ ]:
# Making Single Predection

from tensorflow.keras.preprocessing import image

test_img=image.load_img('/content/drive/MyDrive/classification_dataset/Test/25_Coin/25_1477285914.jpg',target_size=(224,224))
test_img=image.img_to_array(test_img)
test_img=test_img/255

plt.imshow(test_img)

test_img=np.expand_dims(test_img,axis=0)

In [ ]:
reslt=model.predict(test_img)

In [ ]:
reslt

In [ ]:
reslt.argmax(axis=-1)

In [ ]:
a=np.argmax(model.predict(test_img))
a

In [ ]:
# keras.np_utils.probas_to_classes(reslt)

In [ ]:
if a ==0:
  print('The Coin Is: {}'.format(folders[1].split('/')[-1].replace('_Coin','')))
elif a==1:
  print('The Coin Is: {}'.format(folders[2].split('/')[-1].replace('_Coin','')))
elif a==2:
  print('The Coin Is: {}'.format(folders[3].split('/')[-1].replace('_Coin','')))
elif a==3:
  print('the Coin Is: {}'.format(folders[4].split('/')[-1].replace('_Coin','')))
elif a==4 :
  print('The Coin Is: {}'.format(folders[0].split('/')[-1].replace('_Coin',''))) 